In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

def print_lists(LL, names):
    for l_, n_ in zip(LL, names):
        l = n_ + ':'
        for e_ in l_:
            l += "{:5.2f} ".format(e_)
        print (l)

# Ensemble Methods

"Ensemble" is an umbralla term
- progressively build up model capacity w.r.t. fitness to data

- Ensemble: combination of multiple predictors.
- General idea is to construct a set of base predictors (hypotheses) from training data, then to predict unseen data we combine the predictions of the base predictors by voting. 
- Ensemble can be considered as a family of “meta-learning” methods. It is even possible to combine different kinds of data models, such as NN + Decision Trees + … （though a bit unusual).

- Ensemble methods tend to perform better than individual classifiers 
- Ensemble tends to be more reliable

- a bit blurred boundary between the "Algorithm" and "Hypothesis space" in our familiar map of learning framework
<img src="ref/learning.png" width="400"></img>

- Standard Hypothesis space
    - Each $h \in \mathcal H$ is a map $\mathcal X \mapsto \mathcal Y$ (recall the math notions)
    - Learning algorithm consider one $h$ per step
        ```
        while do searching
            compare current h(x_train) with y_train
            update h
        ```

- Ensemble: "weak" hypothesis space $\mathcal H^W$ PLUS hypothesis space $\mathcal H$
    - search (and final model) is in $\mathcal H$
    - each $h \mathcal H$ is a voting scheme by a committee $\{h^w_1, h^w_2, \dots \} \subset \mathcal H^W$

## Case study-1
Suppose we have an ensemble of $m$ (odd) binary classifiers, each of accuracy $\gamma$. The ensemble scheme is a majority vote. Such as:

<img src="ref/3vote.png" width="800"></img>

__Q1__
Compute the ensemble accuracy in the first example given $m$ and $\gamma$. 
- Hint: you can avoid math derivation by adopting empirical evaluation. (example provided)
- Do the curve for more cases of $\gamma=0.51$ and $\gamma=0.9$
- Does trails $N$ have any effect on the result?

In [ ]:
def comp_committee_accu(m, gm, N=1000, randseed=42):
    rng = np.random.RandomState(randseed)
    is_correct = (rng.rand(m, N)<gm).astype(np.float)
    # simulate individual classifier in individual problems
    # if you are not comfortable with the idea of N trails simultaneously
    # imagine N=1 for now.
    
    votes_correct = is_correct.sum(axis=0) # how many committee members are correct 
    corr = (votes_correct > m/2.0).astype(np.float)
    return corr.sum() / float(N)
    

In [ ]:
members = np.arange(1, 200, 2)
accu = [0.6, 0.7, 0.51, 0.9]
accu_curv_1 = [] 
accu_curv_2 = [] 
for mn in members:
    accu_curv_1.append(comp_committee_accu(mn, accu[0]))
    accu_curv_2.append(comp_committee_accu(mn, accu[1]))

In [ ]:
plt.plot(members, accu_curv_1, 'b')
plt.plot(members, accu_curv_2, 'r')

# A (really) Quick Glance of Emsemble Methods

```
while training:
    given current committee_h:{h1, h2, ..., h_k}
```

- Bagging: $h_{k+1} \in \mathcal H^W \leftarrow \mathcal A($  random subset of training data $)$

- Boosting: $h_{k+1} \in \mathcal H^W \leftarrow \mathcal A($  random subset of training data __with weights__ $)$

- Random forest: $h_{k+1} \in \mathcal H^W \leftarrow \mathcal A($  random subset of training data __with weights__ and __selected attributes__ $)$

# AdaBoost
- for $t$ in $\{1, 2, \dots, T\}$
    - $h_t \leftarrow \arg\min_{h \in \mathcal H^W} \big[\epsilon_t(D^t, X)\big]$
        - $\epsilon_t(D_t, X)$: weighted error: $\sum_i D^t_i X_i$
    - weighting $h_t$: $\alpha_t \leftarrow \frac{1}{2}\log \frac{1-\epsilon_t^*}{\epsilon_t^*}$
    - update sample weights: $D'^t_i \leftarrow D^t_i \exp(\pm \alpha_t)$, 
        - then normalise to $D^t_i$


### Two Kinds of Weights
- Weights of Samples: each round we "adjust our focus" on the hard samples.
- Weights of Weak Classifiers: utility of the classifier

### A hand-on example
- X: data
- Y: target values
- D: initial data weights

In [ ]:
X = np.arange(0.1, 1.01, 0.1)
Y = np.asanyarray([1, 1, 1, -1, -1, -1, -1, 1, 1, 1])
D = np.ones_like(Y).astype(float) / X.size
print_lists([X, Y, D], ['X', 'Y', 'D'])

#### Alternative ways of utilising the data weights
1. directly weigting
```
Error = D[i] * Loss(pred[i], Y[i])
```
2. Monte Carlo scheme (see below)

In [ ]:
# Step 1-1 Generating Random Samples
rng = np.random.RandomState(0)
ind = rng.choice(10, size=(10, ), p=D)
ind.sort()
X_ = X[ind]
Y_ = Y[ind]
print_lists([X_, Y_], ['X_', 'Y_'])

In [ ]:
# Let's verify the scheme -- in the limit of large sample size
ind = rng.choice(10, size=(100000, ), p=D)
plt.hist(ind, rwidth=0.9, bins=np.arange(11)-0.5, align='mid')
plt.xlim([-0.7, 9.7])

Let's 'train' a week predictor -- it is no more than a majority vote!

In [ ]:
# Step 1-2 "Training" a week predictor
predictor_1 = lambda x: 1 if x >= 0.75 else -1
output_trn = np.asanyarray(list(map(predictor_1, X_)))
output_1 = np.asanyarray(list(map(predictor_1, X)))
is_correct = np.asanyarray(output_1 == Y).astype(np.float)
print_lists([X_, Y_, output_trn, 
             X, Y, output_1, is_correct], [ 'TrnX', 'TrnY', 'TrnP', '   X', '   Y', 'Pred', 'Corr',])


Assess the outcome:
- $\epsilon_1(D^1, X) := \sum_i D^1_i Loss(Pred_i, Y_i)$ -- weighted error
- $\alpha_1 := 0.5 \log \frac{1-\epsilon_1}{\epsilon_1}$ -- smaller the error, better (and more important) this classifier

In [ ]:
# Step 1-3 Predictor Weight
weighted_errors = (1.0 - is_correct) * D
eps_1 = (weighted_errors).sum()
alpha_1 = 0.5 * np.log( (1-eps_1) / eps_1 )
print_lists([X, D, weighted_errors], ['   X', 'Wght', 'WErr'])

print("Alpha = {:.3f}".format(alpha_1))

Adjust weights according to this round outcome:
- $D^2_i \leftarrow D^1_i \exp(\alpha_1)$: if "difficult" (wrongly classified)
- $\ \ \ \ \  \leftarrow D^1_i \exp(-\alpha_1)$: if "easy"

In [ ]:
# Step 1-4 New sample weights
sample_signs_1 = (0.5 - is_correct) * 2.0
D = D * np.exp(sample_signs_1 * alpha_1)
D /= D.sum()
print_lists([X, D], ['   X', 'NewW'])

In [ ]:
# Let's verify the scheme -- in the limit of large sample size
ind = rng.choice(10, size=(100000, ), p=D)
plt.hist(ind, rwidth=0.9, bins=np.arange(11)-0.5, align='mid')
plt.xlim([-0.7, 10.7])

### ROUND-2 ...

Now let's start over to add a new weak classifier. Note the data have uneven weights now. Draw training samples first, hopefully, we can have those needing improvement over represented this time.

In [ ]:
# Step 2-1 Re draw-samples ...
ind = rng.choice(10, size=(10, ), p=D)
ind.sort()
X_ = X[ind]
Y_ = Y[ind]

print_lists([X_, Y_], ['X_', 'Y_'])

In [ ]:
# Step 2-2 "Training" the 2nd week predictor
predictor_2 = lambda x: 1 if x <=0.35 else -1
output_2 = np.asanyarray(list(map(predictor_2, X)))
is_correct = np.asanyarray(output_2 == Y).astype(np.float)
print_lists([X, Y, output_2, is_correct], ['   X', '   Y', 'Pred', 'Corr'])

Assess the outcome

In [ ]:
# Step 2-3 Predictor Weight
weighted_errors = (1.0 - is_correct) * D
eps_2 = (weighted_errors).sum()
alpha_2 = 0.5 * np.log( (1-eps_2) / eps_2 )
print_lists([X, D, weighted_errors], ['   X', 'Wght', 'WErr'])

print ("Alpha = {:.3f}".format(alpha_2))

Update the sample weights

In [ ]:
# Step 2-4 New sample weights
sample_signs_2 = (0.5 - is_correct) * 2.0
D = D * np.exp(sample_signs_2 * alpha_2)
D /= D.sum()
print_lists([X, D], ['   X', 'NewW'])

Let us test the power of ensemble

In [ ]:
def predict_ensemble(X, ens):
    print("       X:", end="")
    for x_ in X:
        print ('{:5.2f}'.format(x_), end="")
    print()
    
    ens_p = np.zeros_like(X)
    for p, a in ens:
        this_pred_ = np.asanyarray(list(map(p, X)))
        
        ens_p += this_pred_ * a
        
        print ("A={:6.2f}:".format(a), end="")
        for pd_ in this_pred_:
            print ("{:5.2f}".format(pd_*a), end="")
        print()
        
    print ("Final(R):".format(a), end="")
    for pd_ in ens_p:
        print ("{:5.2f}".format(pd_), end="")
    print()
    print ("Final(C):".format(a), end="")
    for pd_ in ens_p:
        print ("{:5.2f}".format(np.sign(pd_)), end="")
    print()

In [ ]:
ensemble = [(predictor_1, alpha_1), 
            (predictor_2, alpha_2)]
predict_ensemble(X, ensemble)

### ROUND-3

In [ ]:
# Step 3-1 Re draw-samples ...
ind = rng.choice(10, size=(10, ), p=D)
ind.sort()
X_ = X[ind]
Y_ = Y[ind]

print_lists([X_, Y_], ['X_', 'Y_'])

In [ ]:
# Step 3-2 "Training" the 3rd week predictor
predictor_3 = lambda x: 1 if x>=0.75 else -1
output_3 = np.asanyarray(list(map(predictor_3, X)))
is_correct = np.asanyarray(output_3 == Y).astype(np.float)
print_lists([X, Y, output_3, is_correct], ['   X', '   Y', 'Pred', 'Corr'])

In [ ]:
# Step 3-3 Predictor Weight
weighted_errors = (1.0 - is_correct) * D
eps_3 = (weighted_errors).sum()
alpha_3 = 0.5 * np.log( (1-eps_3) / eps_3 )
print_lists([X, D, weighted_errors], ['   X', 'Wght', 'WErr'])

print ("Alpha = {:.3f}".format(alpha_3))

In [ ]:
# Step 3-4 New sample weights
sample_signs_3 = (0.5 - is_correct) * 2.0
D = D * np.exp(sample_signs_3 * alpha_3)
D /= D.sum()
print_lists([X, D], ['   X', 'NewW'])

In [ ]:
ensemble = [(predictor_1, alpha_1), 
            (predictor_2, alpha_2),
            (predictor_3, alpha_3)]
predict_ensemble(X, ensemble)

### ROUND-4

In [ ]:
# Step 4-1 Re draw-samples ...
ind = rng.choice(10, size=(10, ), p=D)
ind.sort()
X_ = X[ind]
Y_ = Y[ind]

print_lists([X_, Y_], ['X_', 'Y_'])

In [ ]:
# Step 4-2 "Training" the 4st week predictor
predictor_4 = lambda x: 1 if x < 999 else -1
output_4 = np.asanyarray(list(map(predictor_4, X)))
is_correct = np.asanyarray(output_4 == Y).astype(np.float)
print_lists([X, Y, output_4, is_correct], ['   X', '   Y', 'Pred', 'Corr'])

In [ ]:
# Step 4-3 Predictor Weight
weighted_errors = (1.0 - is_correct) * D
eps_4 = (weighted_errors).sum()
alpha_4 = 0.5 * np.log( (1-eps_4) / eps_4 )
print_lists([X, D, weighted_errors], ['   X', 'Wght', 'WErr'])

print ("Alpha = {:.3f}".format(alpha_4))

In [ ]:
# Step 4-4 New sample weights
sample_signs_4 = (0.5 - is_correct) * 2.0
D = D * np.exp(sample_signs_4 * alpha_4)
D /= D.sum()
print_lists([X, D], ['   X', 'NewW'])

In [ ]:
ensemble = [(predictor_1, alpha_1), 
            (predictor_2, alpha_2),
            (predictor_3, alpha_3),
            (predictor_4, alpha_4),]
predict_ensemble(X, ensemble)

# LAB

We consider building simple classifers on 2D points to approximate interesting data distributions. Under "Code Adaboost". Some preparation / helper codes are provided. The answer is provided (commented out block), but it is recommended to try yourself before check with the provided code.

In [ ]:
import numpy as np
# Generate data
SAMPLE_NUM_PER_CLASS = 200
rng = np.random.RandomState(0)
X0  = rng.randn(SAMPLE_NUM_PER_CLASS, 2) * 0.15
X1_a  = rng.rand(SAMPLE_NUM_PER_CLASS) * 2 * np.pi
X1_r  = rng.rand(SAMPLE_NUM_PER_CLASS) * 0.2 + 0.4
X1 = np.asarray([np.cos(X1_a)*X1_r, np.sin(X1_a)*X1_r]).T
X = np.concatenate((X0, X1), axis=0)
y = np.concatenate((np.zeros(SAMPLE_NUM_PER_CLASS), 
                    np.ones(SAMPLE_NUM_PER_CLASS)), axis=0)

In [ ]:
plt.scatter(X[:,0], X[:,1], c=y, s=32, cmap='summer', edgecolor='k')

In [ ]:
# function / return value / list / python-construction of list / product
# Define weak hypothesis

class FlatCut:
    """
    2D flat lines to cut a squre
    """
    def __init__(self):
        self.mode = 'Undetermined'
        self.th = None
        
    def predict(self, data):
        if self.mode == 'horizontal_gt':
            pred = data[:, 0] >= self.th
        elif self.mode == 'horizontal_lt':
            pred = data[:, 0] < self.th
        elif self.mode == 'vertical_gt':
            pred = data[:, 1] >= self.th
        elif self.mode == 'vertical_lt':
            pred = data[:, 1] < self.th
        else:
            assert False, "Unknown mode "
            
        return pred.astype(np.float) 
            
    def fit(self, data, targets):
        self.target_values = np.unique(targets)
        assert self.target_values.size <= 2, "Deal with 2-class problem only"
        
        if self.target_values.size == 1:
            self.mode = 'horizontal_gt'
            self.th = - inf # dummy prediction
        
        # "moving knife" cutting at some x-pos
        xmin, xmax = data[:, 0].min(), data[:, 0].max()
        ymin, ymax = data[:, 1].min(), data[:, 1].max()
        best_th = None
        best_mode = None
        best_accuracy = 0
        for self.mode in ['horizontal_gt', 'horizontal_lt']:
            for self.th in np.linspace(xmin, xmax, 100):
                accu = np.count_nonzero(self.predict(data)==targets) / float(targets.size)
                if accu > best_accuracy:
                    best_mode = self.mode
                    best_th = self.th
                    best_accuracy = accu
                    
        for self.mode in ['vertical_gt', 'vertical_lt']:
            for self.th in np.linspace(ymin, ymax, 100):
                accu = np.count_nonzero(self.predict(data)==targets) / float(targets.size)
                if accu > best_accuracy:
                    best_mode = self.mode
                    best_th = self.th
                    best_accuracy = accu
                
                
        self.th = best_th
        self.mode = best_mode
        print (self.mode, self.th)

In [ ]:
wc = FlatCut()
wc.fit(X, y)
p = wc.predict(X)
plt.scatter(X[:,0], X[:,1], c=p, s=32, cmap='summer', edgecolor='k')
plt.show()

In [ ]:
def ensemble_pred(X, ensemble):
    N = X.shape[0]
    A0 = np.zeros(N)
    A1 = np.zeros(N)
    
    for p, a in ensemble:
        pred = p.predict(X)
        A0[pred==0] += a
        A1[pred==1] += a
        
    return (A1>A0).astype(float)

## Code Adaboost

In [ ]:
# AdaBoost
rng = np.random.RandomState(0)
N = X.shape[0]
subsample_size = N
T = 50
D = np.ones(N)/X.shape[0] # initial distribution
ind = rng.choice(X.shape[0], size=(subsample_size,), p=D)
X_ = X[ind]
y_ = y[ind]
ensemble = []
for t in range(T):
    
    weak_pred_t = FlatCut()
    weak_pred_t.fit(X_, y_) # RANSAC SimpleModel: Y -> X' ~ X 

    # ANSWER #
#     pred = weak_pred_t.predict(X)
#     errors = (pred != y).astype(np.float)
#     errors_w = errors * D
#     eps_t = max(errors_w.sum(), 1e-6)           # to avoid singular division
    
#     #  compute hypothesis(predictor) weight
#     alpha = 0.5 * np.log((1-eps_t) / eps_t)
    
#     D *= np.exp( (errors - 0.5) * 2.0 * alpha ) # adjust sample weights 
#     D /= D.sum()                                # normalise to 1
    
#     # add to ensemble
#     ensemble.append((weak_pred_t, alpha))
    

    # visualise
    plt.figure(1, (12, 4))
    plt.subplot(1,3,1)
    plt.scatter(X_[:,0], X_[:,1], c=weak_pred_t.predict(X_), cmap='summer', s=64, edgecolor='k')
    plt.subplot(1,3,2)
    plt.scatter(X[:,0], X[:,1], c=ensemble_pred(X, ensemble), cmap='summer', s=64, vmin=0, vmax=1, edgecolor='k')
    plt.subplot(1,3,3)
    plt.scatter(X[:,0], X[:,1], c=D, cmap='hot', s=64, vmin=0, vmax=D.max(), edgecolor='k')
    plt.show()
    
    
    ind = rng.choice(X.shape[0], size=(subsample_size,), p=D)
    
    X_ = X[ind]
    y_ = y[ind]
    

